## 1. Keyword/Text search with Korean Analyzer

### Analyzer

- https://learn.microsoft.com/en-us/azure/search/search-analyzers
- https://learn.microsoft.com/en-us/azure/search/index-add-language-analyzers

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from azure.core.credentials import AzureKeyCredential  
from azure.search.documents import SearchClient  
from azure.search.documents.indexes import SearchIndexClient  
from azure.search.documents.indexes.models import (  
    SearchIndex,  
    SearchField,  
    SearchFieldDataType,  
    SimpleField,  
    SearchableField,  
    SearchIndex,  
)

In [3]:
import os
service_endpoint = os.getenv("AZSCH_ENDPOINT")  
credential = AzureKeyCredential(os.environ["AZSCH_KEY"])

#print(service_endpoint)

In [4]:
# Create a search index
def create_search_index(index_name):
    index_client = SearchIndexClient(
        endpoint=service_endpoint, credential=credential)
    fields = [
        SimpleField(name="id", type=SearchFieldDataType.String, key=True),
        SearchableField(name="proverb", type=SearchFieldDataType.String,
                        searchable=True, retrievable=True,
                        analyzer_name="ko.microsoft"),
        SearchableField(name="meaning", type=SearchFieldDataType.String,
                        searchable=True, retrievable=True,
                        analyzer_name="ko.microsoft"),
        SearchableField(name="source", type=SearchFieldDataType.String,
                        searchable=False, retrievable=True,
                        facetable=True, filterable=True),
        SearchableField(name="type", type=SearchFieldDataType.String,
                        searchable=False, retrievable=True,
                        facetable=True, filterable=True),
    ]

    # Create the search index
    index = SearchIndex(name=index_name, fields=fields)
    result = index_client.create_or_update_index(index)
    print(f' {result.name} created')

In [5]:
index_name = "korproverb-keyword-index"
create_search_index(index_name)

 korproverb-keyword-index created


In [6]:
search_client = SearchClient(endpoint=service_endpoint, index_name=index_name, credential=credential)

In [7]:
import pandas as pd
from tqdm import tqdm

In [8]:
df = pd.read_csv('./source/idioms.csv')
df.head()

,Description,Meaning,Source,Type
0,가갸 뒤 자〔뒷다리〕도 모른다,"글자를 모르는 사람을 비유적으로 이르는 말.,사리에 어두운 사람을 놀림조로 이르는 말.",고려대 한국어대사전,속담
1,가까운 남이 먼 일가보다 낫다,이웃끼리 서로 친하게 지내다 보면 먼 곳에 있는 일가보다 더 친하게 되어 서로 도우...,표준국어대사전,속담
2,가까운 데 집은 깎이고 먼 데 절은 비친다,가까운 데 있는 절이나 집은 자잘한 흠도 잘 드러나서 좋지 않아 보이고 먼 곳에 윤...,표준국어대사전,속담
3,가까운 무당보다 먼 데 무당이 영하다,흔히 사람은 자신이 잘 알고 가까이 있는 것보다는 잘 모르고 멀리 있는 것을 더 좋...,표준국어대사전,속담
4,가까운 집 며느리일수록 흉이 많다,늘 가까이 있고 잘 아는 사이일수록 상대편의 결점이 눈에 더 많이 띈다는 말.,표준국어대사전,속담


In [9]:
count = 0
batch_size = 40
for i in tqdm(range(0, len(df), batch_size)):
    # set end position of batch
    i_end = min(i+batch_size, len(df))
    
    documents = df[i:i_end].apply(
        lambda row: {'id': str(row.name), 
                     'proverb': row['Description'], 
                     'meaning': row['Meaning'],
                     'source': row['Source'],
                     'type': str(row['Type'])
                    }, axis=1).to_list()
    
    result = search_client.upload_documents(documents)  

100%|██████████| 164/164 [00:32<00:00,  5.10it/s]


In [10]:
from azure.search.documents.models import (
    QueryType,
    SearchMode
)

import json

def azsch_text_query(query, type=QueryType.Simple, mode=SearchMode.Any):

    results = search_client.search(  
        search_text=query,
        query_type=type,
        search_mode=mode,
        search_fields=["meaning"],
        select=["proverb", "meaning", "source", "type"],
        query_language="ko-kr",
        top=10 # for limiting text search
    ) 
    
    print("Search Results:")
    for i, result in enumerate(results, 1): 
        print(f"{i}: {result['@search.score']:.10f}: {result['proverb']} - {result['meaning']}, {result['type']}")  


## Demo

- basic keyword search
- filter
- paging
- faceting

### basic search

In [11]:
results = azsch_text_query('노력')

Search Results:
1: 7.9132340000: 몸부림 치다 - (사람이) 온갖 노력을 다하다., 관용구
2: 7.5999850000: 피땀 흘리다 - 온갖 힘과 정성을 쏟아 노력하다., 관용구
3: 6.5693436000: 소불알 떨어지면 구워 먹겠다고 소금 가지고 따라다닌다 - 노력은 안 하고 산 소의 불알이 저절로 떨어지기를 마냥 기다리기만 한다는 뜻으로, 노력 없이 요행만 바라는 헛된 짓을 비유적으로 이르는 말., 속담
4: 6.0504380000: 양반은 물에 빠져도 개헤엄은 안 한다 - 아무리 위급한 때라도 체면을 유지하려고 노력한다는 말., 속담
5: 5.9048786000: 하늘에서  떨어지다 - (어떠한 성과가) 별로 노력을 들이지 않았는데 저절로 얻어지다., 관용구
6: 5.8381233000: 내 침 발라 꼰 새끼가 제일 - 자기의 노력을 들여 이룩한 성과가 귀중함을 비유적으로 이르는 말., 속담
7: 5.8216457000: 새도 날려면 움츠린다 - 무슨 일이든 사전에 준비할 틈과 노력이 필요하다는 말. (=개구리도 옴쳐야 뛴다.), 속담
8: 5.7526180000: 둔한 말이 열 수레를 끈다 - 미련하고 둔한 사람이라 하더라도 쉬지 않고 노력하면 큰일을 할 수 있다는 말., 속담
9: 5.6404715000: 눈을 떠야 별을 보지 - 어떤 성과를 거두려면 그에 상당한 노력과 준비가 있어야 한다는 말., 속담
10: 5.6404715000: 체메 들다 - 남의 사정이나 수단에 의하여 어이없이 돈이나 노력을 대신 부담하다., 관용구


In [12]:
azsch_text_query('노력 준비')

Search Results:
1: 12.2485750000: 눈을 떠야 별을 보지 - 어떤 성과를 거두려면 그에 상당한 노력과 준비가 있어야 한다는 말., 속담
2: 11.0136430000: 새도 날려면 움츠린다 - 무슨 일이든 사전에 준비할 틈과 노력이 필요하다는 말. (=개구리도 옴쳐야 뛴다.), 속담
3: 10.2823400000: 잠을 자야 꿈을 꾸지 - 어떤 성과를 거두려면 그에 상당한 노력과 준비가 있어야 한다는 말.,원인 없이 결과를 바랄 수 없음을 이르는 말., 속담
4: 9.4363500000: 하늘을 보아야 별을 따지 - 어떤 성과를 거두려면 그에 상당한 노력과 준비가 있어야 한다는 말.,무슨 일이 이루어질 기회나 조건이 전혀 없음을 이르는 말., 속담
5: 7.9132340000: 몸부림 치다 - (사람이) 온갖 노력을 다하다., 관용구
6: 7.7247815000: 노루 본 놈이〔노루 보고〕 그물 짊어진다 - 무슨 일을 미리 준비하지 않고 일을 당해서야 허겁지겁 준비함을 비유적으로 이르는 말., 속담
7: 7.5999850000: 피땀 흘리다 - 온갖 힘과 정성을 쏟아 노력하다., 관용구
8: 7.0838730000: 신들메를 조이다 - 어떤 일을 할 준비를 든든히 하다., 관용구
9: 6.5693436000: 소불알 떨어지면 구워 먹겠다고 소금 가지고 따라다닌다 - 노력은 안 하고 산 소의 불알이 저절로 떨어지기를 마냥 기다리기만 한다는 뜻으로, 노력 없이 요행만 바라는 헛된 짓을 비유적으로 이르는 말., 속담
10: 6.4667630000: 이도 안 나다 - 어떤 일을 감당하기에는 수준이나 준비 정도가 적당하지 않다., 관용구


In [13]:
azsch_text_query('+노력 +준비')

Search Results:
1: 12.2485750000: 눈을 떠야 별을 보지 - 어떤 성과를 거두려면 그에 상당한 노력과 준비가 있어야 한다는 말., 속담
2: 11.0136430000: 새도 날려면 움츠린다 - 무슨 일이든 사전에 준비할 틈과 노력이 필요하다는 말. (=개구리도 옴쳐야 뛴다.), 속담
3: 10.2823400000: 잠을 자야 꿈을 꾸지 - 어떤 성과를 거두려면 그에 상당한 노력과 준비가 있어야 한다는 말.,원인 없이 결과를 바랄 수 없음을 이르는 말., 속담
4: 9.4363500000: 하늘을 보아야 별을 따지 - 어떤 성과를 거두려면 그에 상당한 노력과 준비가 있어야 한다는 말.,무슨 일이 이루어질 기회나 조건이 전혀 없음을 이르는 말., 속담


In [14]:
azsch_text_query('열심히 노력하면 성공한다는 말')

Search Results:
1: 14.8988640000: 누구는 날 때부터 안다더냐 - 사람이면 누구나 똑같으므로 열심히 노력하면 배울 수 있다는 말., 속담
2: 12.1209720000: 십 년 적공이면 한 가지 성공을 한다 - 무슨 일이든지 오랫동안 꾸준히 노력하면 마침내는 성공하게 됨을 이르는 말., 속담
3: 8.0702250000: 둔한 말이 열 수레를 끈다 - 미련하고 둔한 사람이라 하더라도 쉬지 않고 노력하면 큰일을 할 수 있다는 말., 속담
4: 7.9132340000: 몸부림 치다 - (사람이) 온갖 노력을 다하다., 관용구
5: 7.5999850000: 피땀 흘리다 - 온갖 힘과 정성을 쏟아 노력하다., 관용구
6: 7.5652870000: 눈과 귀가 쏠리다 - 마음이 끌리어 열심히 듣거나 보다., 관용구
7: 7.4523780000: 절뚝발이 말이 천리 간다 - 누구나 꾸준히 노력하면 무슨 일이든 이룰 수 있다는 말. (=둔한 말도 열흘 가면 천리를 간다.), 속담
8: 7.1888440000: 소매가 길면 춤을 잘 추고 돈이 많으면 장사를 잘한다 - 수단이나 밑천이 든든하면 성공하기 쉽다는 말., 속담
9: 7.1469235000: 쥐도 한몫 보면 락이 있다 - 한길로 전심전력하면 성공할 때가 있음을 비유적으로 이르는 말., 속담
10: 6.8657455000: 소불알 떨어지면 구워 먹겠다고 소금 가지고 따라다닌다 - 노력은 안 하고 산 소의 불알이 저절로 떨어지기를 마냥 기다리기만 한다는 뜻으로, 노력 없이 요행만 바라는 헛된 짓을 비유적으로 이르는 말., 속담


In [15]:
azsch_text_query('열심히 노력하면 +성공')

Search Results:
1: 11.7557430000: 십 년 적공이면 한 가지 성공을 한다 - 무슨 일이든지 오랫동안 꾸준히 노력하면 마침내는 성공하게 됨을 이르는 말., 속담


### Advanced search

- Facet
- Filter

In [16]:
def azsch_text_query_adv(query, type_filter="", skip=0):

    results = search_client.search(  
        search_text=query,
        query_type=QueryType.Simple,
        search_mode=SearchMode.Any,
        search_fields=["meaning"],
        select=["proverb", "meaning", "source", "type"],
        facets=["source", "type"],
        query_language="ko-kr",
        include_total_count=True,
        skip=skip,
        filter=None if type_filter == "" else f"type eq '{type_filter}'",
        top=10 # for limiting text search
    ) 
    
    print("Search Results:")
    print(f"- count: {results.get_count()}")
    print(f"- facets: {json.dumps(results.get_facets(), indent=4, ensure_ascii=False)}\n")
    for i, result in enumerate(results, 1):  
        print(f"{i}: {result['@search.score']:.10f}: {result['proverb']} - {result['meaning']}, {result['type']}")  

In [17]:
# facets
azsch_text_query_adv("노력")

Search Results:
- count: 42
- facets: {
    "type": [
        {
            "value": "속담",
            "count": 32
        },
        {
            "value": "관용구",
            "count": 10
        }
    ],
    "source": [
        {
            "value": "표준국어대사전",
            "count": 22
        },
        {
            "value": "고려대 한국어대사전",
            "count": 17
        },
        {
            "value": "우리말샘",
            "count": 3
        }
    ]
}

1: 7.9155335000: 몸부림 치다 - (사람이) 온갖 노력을 다하다., 관용구
2: 7.1347523000: 피땀 흘리다 - 온갖 힘과 정성을 쏟아 노력하다., 관용구
3: 6.7460810000: 힘을 돌리다 - 어떤 일을 이루려고 거기에 노력을 들이다., 관용구
4: 6.5704170000: 소불알 떨어지면 구워 먹겠다고 소금 가지고 따라다닌다 - 노력은 안 하고 산 소의 불알이 저절로 떨어지기를 마냥 기다리기만 한다는 뜻으로, 노력 없이 요행만 바라는 헛된 짓을 비유적으로 이르는 말., 속담
5: 6.0549674000: 양반은 물에 빠져도 개헤엄은 안 한다 - 아무리 위급한 때라도 체면을 유지하려고 노력한다는 말., 속담
6: 5.9218345000: 하늘에서  떨어지다 - (어떠한 성과가) 별로 노력을 들이지 않았는데 저절로 얻어지다., 관용구
7: 5.8526664000: 내 침 발라 꼰 새끼가 제일 - 자기의 노력을 들여 이룩한 성과가 귀중함을 비유적으로 이르는 말., 속담
8: 5.8267465000: 힘을 넣다 - 어떤 일을 이루

In [18]:
# paging
azsch_text_query_adv("노력", "", 10)

Search Results:
- count: 42
- facets: {
    "type": [
        {
            "value": "속담",
            "count": 32
        },
        {
            "value": "관용구",
            "count": 10
        }
    ],
    "source": [
        {
            "value": "표준국어대사전",
            "count": 22
        },
        {
            "value": "고려대 한국어대사전",
            "count": 17
        },
        {
            "value": "우리말샘",
            "count": 3
        }
    ]
}

1: 5.6500770000: 눈을 떠야 별을 보지 - 어떤 성과를 거두려면 그에 상당한 노력과 준비가 있어야 한다는 말., 속담
2: 5.6500770000: 체메 들다 - 남의 사정이나 수단에 의하여 어이없이 돈이나 노력을 대신 부담하다., 관용구
3: 5.5639530000: 겉보리 한 말 주고 푸닥거리하기보다 낫다 - 들인 노력에 비하여 결과가 만족스럽다는 것을 비유적으로 이르는 말., 속담
4: 5.5639530000: 누구는 날 때부터 안다더냐 - 사람이면 누구나 똑같으므로 열심히 노력하면 배울 수 있다는 말., 속담
5: 5.5559607000: 구르는 돌은 이끼가 안 낀다 - 부지런하고 꾸준히 노력하는 사람은 침체되지 않고 계속 발전한다는 말., 속담
6: 5.5559607000: 도 닦다 - (사람이) 어떤 분야나 영역에 정통하고자 능력을 쌓기 위하여 노력하다., 관용구
7: 5.2419950000: 굴을 파야 금을 얻는다 - 목적을 이루기 위하여서는 거기에 필요한 조건을 갖추거나 노력을 하여야 함을 교훈적으로 이르는 말., 속담
8: 5

In [19]:
# filter
azsch_text_query_adv('노력', "관용구")

Search Results:
- count: 10
- facets: {
    "type": [
        {
            "value": "관용구",
            "count": 10
        }
    ],
    "source": [
        {
            "value": "고려대 한국어대사전",
            "count": 5
        },
        {
            "value": "표준국어대사전",
            "count": 5
        }
    ]
}

1: 7.9155335000: 몸부림 치다 - (사람이) 온갖 노력을 다하다., 관용구
2: 7.1347523000: 피땀 흘리다 - 온갖 힘과 정성을 쏟아 노력하다., 관용구
3: 6.7460810000: 힘을 돌리다 - 어떤 일을 이루려고 거기에 노력을 들이다., 관용구
4: 5.9218345000: 하늘에서  떨어지다 - (어떠한 성과가) 별로 노력을 들이지 않았는데 저절로 얻어지다., 관용구
5: 5.8267465000: 힘을 넣다 - 어떤 일을 이루려고 시간, 노력, 자금 따위를 투입하다., 관용구
6: 5.6500770000: 체메 들다 - 남의 사정이나 수단에 의하여 어이없이 돈이나 노력을 대신 부담하다., 관용구
7: 5.5559607000: 도 닦다 - (사람이) 어떤 분야나 영역에 정통하고자 능력을 쌓기 위하여 노력하다., 관용구
8: 5.1356535000: 빈손 털다 - (사람이) 들인 노력이 헛일이 되어 아무 소득이 없다.,(사람이) 가지고 있던 것을 몽땅 털다., 관용구
9: 5.0807540000: 손 붙이다 - (사람이) 어떤 일을 시작하다.,(사람이) 모자란 일손을 채우거나 노력을 들여 일하다., 관용구
10: 4.8938146000: 열 번 죽었다 깨어도 - 열 번 죽었다가 살아날 만큼 온갖 노력을 다하여도.,열 번 죽었다 살아날 만큼 어려운 일을 당하여도., 관용구


### Hightlight

In [20]:
def azsch_text_query_highlight(query):

    results = search_client.search(  
        search_text=query,
        query_type=QueryType.Simple,
        search_mode=SearchMode.Any,
        search_fields=["meaning"],
        select=["proverb", "meaning", "source", "type"],
        highlight_fields="meaning",
        query_language="ko-kr",
        top=10 # for limiting text search
    ) 
    
    print("Search Results:")
    for i, result in enumerate(results, 1):  
        if (result["@search.highlights"]):
            print(f"{i}: {result['@search.score']:.10f}: {result['proverb']} - {result['@search.highlights']['meaning'][0]}, {result['type']}")
        else:
            print(f"{i}: {result['@search.score']:.10f}: {result['proverb']} - {result['meaning']}, {result['type']}")  

In [21]:
# facets
azsch_text_query_highlight("노력")

Search Results:
1: 7.9155335000: 몸부림 치다 - (사람이) 온갖 <em>노력을</em> 다하다., 관용구
2: 7.1347523000: 피땀 흘리다 - 온갖 힘과 정성을 쏟아 <em>노력</em>하다., 관용구
3: 6.7460810000: 힘을 돌리다 - 어떤 일을 이루려고 거기에 <em>노력을</em> 들이다., 관용구
4: 6.5704170000: 소불알 떨어지면 구워 먹겠다고 소금 가지고 따라다닌다 - <em>노력은</em> 안 하고 산 소의 불알이 저절로 떨어지기를 마냥 기다리기만 한다는 뜻으로, <em>노력</em> 없이 요행만 바라는 헛된 짓을 비유적으로 이르는 말., 속담
5: 6.0549674000: 양반은 물에 빠져도 개헤엄은 안 한다 - 아무리 위급한 때라도 체면을 유지하려고 <em>노력</em>한다는 말., 속담
6: 5.9218345000: 하늘에서  떨어지다 - (어떠한 성과가) 별로 <em>노력을</em> 들이지 않았는데 저절로 얻어지다., 관용구
7: 5.8526664000: 내 침 발라 꼰 새끼가 제일 - 자기의 <em>노력을</em> 들여 이룩한 성과가 귀중함을 비유적으로 이르는 말., 속담
8: 5.8267465000: 힘을 넣다 - 어떤 일을 이루려고 시간, <em>노력</em>, 자금 따위를 투입하다., 관용구
9: 5.8226060000: 새도 날려면 움츠린다 - 무슨 일이든 사전에 준비할 틈과 <em>노력이</em> 필요하다는 말., 속담
10: 5.7535640000: 둔한 말이 열 수레를 끈다 - 미련하고 둔한 사람이라 하더라도 쉬지 않고 <em>노력</em>하면 큰일을 할 수 있다는 말., 속담


In [22]:
azsch_text_query_highlight("노력 준비")

Search Results:
1: 12.2678930000: 눈을 떠야 별을 보지 - 어떤 성과를 거두려면 그에 상당한 <em>노력과</em> <em>준비가</em> 있어야 한다는 말., 속담
2: 11.0158630000: 새도 날려면 움츠린다 - 무슨 일이든 사전에 <em>준비</em>할 틈과 <em>노력이</em> 필요하다는 말., 속담
3: 10.0445350000: 잠을 자야 꿈을 꾸지 - 어떤 성과를 거두려면 그에 상당한 <em>노력과</em> <em>준비가</em> 있어야 한다는 말., 속담
4: 9.4706210000: 하늘을 보아야 별을 따지 - 어떤 성과를 거두려면 그에 상당한 <em>노력과</em> <em>준비가</em> 있어야 한다는 말., 속담
5: 7.9155335000: 몸부림 치다 - (사람이) 온갖 <em>노력을</em> 다하다., 관용구
6: 7.7387890000: 노루 본 놈이〔노루 보고〕 그물 짊어진다 - 무슨 일을 미리 <em>준비</em>하지 않고 일을 당해서야 허겁지겁 <em>준비</em>함을 비유적으로 이르는 말., 속담
7: 7.1347523000: 피땀 흘리다 - 온갖 힘과 정성을 쏟아 <em>노력</em>하다., 관용구
8: 7.1078250000: 신들메를 조이다 - 어떤 일을 할 <em>준비를</em> 든든히 하다., 관용구
9: 6.7460810000: 힘을 돌리다 - 어떤 일을 이루려고 거기에 <em>노력을</em> 들이다., 관용구
10: 6.5704170000: 소불알 떨어지면 구워 먹겠다고 소금 가지고 따라다닌다 - <em>노력은</em> 안 하고 산 소의 불알이 저절로 떨어지기를 마냥 기다리기만 한다는 뜻으로, <em>노력</em> 없이 요행만 바라는 헛된 짓을 비유적으로 이르는 말., 속담
